# Pixel Drill for Indices outputs to CSV

| Authors:  | Bex Dunn|
|----------|----------------|
| Created: | March 6, 2019 |
| Last edited: | March 6, 2019 |

**Requirements:**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

__Background:__ Data from the [Landsat](https://landsat.usgs.gov/about-landsat) 5,7 and 8 satellite missions are accessible through [Digital Earth Australia](http://www.ga.gov.au/about/projects/geographic/digital-earth-australia) (DEA).

__What does this notebook do?:__ This notebook takes a supplied CSV of site points. It runs a pixel drill through surface reflectance, calculates NDVI, Taselled cap wetness and greenness, and outputs a csv of values for each site.

import some modules

In [1]:
import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
import shapely
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry

sys.path.append('../10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, BandIndices

dc = datacube.Datacube(app='pixel drill')

%load_ext autoreload

%autoreload 2

### set up file to open 

In [2]:
inpath = '/g/data/r78/rjd547/LTIM_veg_monitoring/LTIM vegetation monitoring sites.csv'

### Convert csv latitude and longitude values into a geopandas geodatafrome

In [3]:
veg_sites = pd.read_csv(inpath, delimiter=",")
#turn csv geometry into geopandas dataframe geometry using lambda functions and shapely
veg_sites['geometry']=veg_sites.apply(lambda z: shapely.geometry.Point(z.LATITUDE, z.LONGITUDE), axis=1)
veg_sites = gpd.GeoDataFrame(veg_sites)

In [4]:
#define an output location
output_loc = '/g/data/r78/rjd547/LTIM_veg_monitoring/'

In [5]:
veg_sites.head(2)

,PROGRAM,NAME,DESCRIPTION,LONGITUDE,LATITUDE,Data collected,sampling dates 2014,sampling dates 2015,sampling dates 2016,sampling dates 2017,sampling dates 2018,geometry
0,",Gwydir River System,",GWY_MUNG1,Mungwonga 1,149.292034,-29.233837,"veg community structure, Veg Abundance,",18/11/2014,12/03/2015\n15/10/2015\n,14/03/2016\n28/10/2016\n,6/03/2017,NaN,POINT (-29.23383652 149.292034)
1,",Gwydir River System,",GWY_ODB1,Old Dromana Bolboshc,149.299029,-29.341779,"veg community structure, Veg Abundance,",NaN,NaN,NaN,NaN,NaN,POINT (-29.34177914 149.299029)


### use a test index to try running this for only one site

In [6]:
test_index=0

### use index of site to get query information

In [7]:
query = {'lat':(veg_sites['LATITUDE'][test_index]), 
         'lon':(veg_sites['LONGITUDE'][test_index])}               

### get name of site from heading of column

In [8]:
Veg_Site = '_'.join(veg_sites['NAME'][test_index].split(' '))
print(Veg_Site)

GWY_MUNG1


In [9]:
outfilename=output_loc+Veg_Site+'.csv'
print (outfilename)

/g/data/r78/rjd547/LTIM_veg_monitoring/GWY_MUNG1.csv


In [10]:
ls578 = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart')

Loading ls5
    Loading 338 filtered ls5 timesteps
Loading ls7
    Ignoring SLC-off observations for ls7
    Loading 73 filtered ls7 timesteps
Loading ls8
    Loading 131 filtered ls8 timesteps
Combining and sorting ls5, ls7, ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)


### Calculate NDVI 

### Calculate the tasseled cap indices

In [11]:
#transform the nbart into tci - we want unthresholded, which is why we set drop_tci_bands to False
tci = TasseledCapTools.thresholded_tasseled_cap(ls578,wetness_threshold=-350, drop=True , drop_tc_bands=False)

In [12]:
tcw = tci.wetness
tcg = tci.greenness

In [13]:
ndvi = BandIndices.calculate_indices(ls578, index='NDVI')

The formula we are using is (nir - red)/(nir + red)


create and fill pandas dataframe to write to csv

In [14]:
#drop dimensions of length 1
ndvi = ndvi.squeeze('x')
ndvi = ndvi.squeeze('y')

In [15]:
#make a new dataframe using the data from the xarray of ndvi
NDVI = pd.DataFrame(data=ndvi.data, index=ndvi.time,columns=['ndvi'])

In [16]:
#rename the dataframe so that we can add the other indexes
INDICIES=NDVI

In [17]:
#again drop dims
TCW = tcw.squeeze().data
TCG = tcg.squeeze().data

In [18]:
# set up final dataframe to write to file 'INDICES'

In [19]:
INDICIES['tcw']=TCW
INDICIES['tcg']=TCG
#we have some values where the pixel is cloudy. Drop these values
INDICIES = INDICIES.dropna(axis='index', thresh =2)

In [20]:
INDICIES.to_csv(outfilename)

In [21]:
INDICIES.head()

,ndvi,tcw,tcg
time,,,
1986-08-21 23:16:16.500,0.282067,-3263.1508,276.9573
1987-09-09 23:21:35.500,0.488089,-2548.1117,1033.1784
1987-09-25 23:21:58.000,0.428260,-2480.8821,884.0466
1987-10-11 23:22:17.000,0.407141,-3039.4717,766.4563
1987-11-28 23:23:11.000,0.267192,-2302.3268,401.8017
